# Domain Deployment Tutorial

By the end of this tutorial, you will have launched a domain server and loaded in private data. This will enable a data scientist somewhere else to use your data for data science without being able to download it.

- **Section 1**: Setup your Notebook
- **Section 2**: Install Dependencies
- **Section 3**: Launch Server
- **Section 4**: Upload Data to your server
- **Section 5**: Join a Network
- **Section 6**: Create a Data Scientist account
- **Section 7**: Conclusion

# BEGIN SECTION 1: SETUP YOUR NOTEBOOK

### Step 0: Instructions

Every time you see the image below, it means we want to you to wait as a concept will be explained at this step.

<img src="img/tab_info.png" alt="tab" style="width: 100px; margin:0;" />

Every time you see the `RUN` image below, it means we want to you to run the cell below the image.

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

### Step 1: Verify you have your Session Details

<img src="img/tab_info.png" alt="tab" style="width: 100px; margin:0;" />


**Example Details**

```
Hi Person,
These are your Session Details:
-------------------------------
VM Username: azureuser
VM Password: **********
VM IP Address: x.x.x.x

📎 MY_DATASET_URL:
https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/subsets/xxxxxxx.pkl
```

### Step 2: Learn to run a cell in the notebook

To run a cell either click on the ▶ button (top left hand corner) or press `Shift + Enter` from your keyboard.

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
print("Hello World !!!")

### Step 3: 🖐 Raise your hand in Zoom

# BEGIN SECTION 2: INSTALL DEPENDENCIES

### Step 4: Install the latest version of "pip"

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
%%capture pip
!sudo apt update && sudo apt install python3-pip
print("pip is successfully installed !!!")

### Step 5: Install the latest version of "hagrid"

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
%%capture pip_hagrid
!pip install -U hagrid
print("Hagrid is successfully installed !!!")

### Step 6: Install the latest version of "PySyft"

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
%%capture syft
!pip install -q --pre syft
print("Syft is successfully installed !!!")

### Step 7: 🖐 Raise your hand in Zoom

# BEGIN SECTION 3: LAUNCH SERVER

### Step 8: Use hagrid to launch your domain node server


The HAGrid launch command follows the following format:
```bash
    hagrid launch <node_type> <node_name> to <target_host>
```

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
%%capture hagrid
!hagrid launch to docker:80 --tag=latest --tail=false

### Step 9: Check the health of your domain node

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
!hagrid check --wait --silent

### Step 10: 🖐 Raise your hand in Zoom

# BEGIN SECTION 4: UPLOAD DATA TO YOUR PRIVATE DATA SERVER

### Step 11: Import Syft Library

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
import syft as sy

print(f"You're running syft version: {sy.__version__}")

### Step 12: Set Login credentials

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
from utils import *

DOMAIN_HOST_IP = auto_detect_domain_host_ip()
ADMIN_EMAIL="info@openmined.org"
ADMIN_PASSWORD="changethis"

print("💳 Login Credentials are set !!!")

### Step 13: Use Syft library to log into your Domain

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client = sy.login(url=DOMAIN_HOST_IP, email=ADMIN_EMAIL, password=ADMIN_PASSWORD)

### Step 14: Your Unique Dataset

<img src="img/tab_info.png" alt="tab" style="width: 100px; margin:0;" />


You have been provided a link to your unique dataset in your session details.

TODO: Add an image highlighting the dataset url.

### Step 15: Copy and Paste dataset URL from your Session Details

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
MY_DATASET_URL = ""
print("My Dataset URL: ", MY_DATASET_URL)

### Step 16: Download Dataset

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
dataset = download_tissue_mnist_dataset(MY_DATASET_URL)

### Step 17: Glance over the downloaded dataset

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# Let's get a peek of the dataset
dataset.head()

### Step 18: Split and preprocess dataset

In [ ]:
train, val, test = split_and_preprocess_dataset(data=dataset)

### Step 19: Make Training data private

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
data_subjects = DataSubjectList.from_series(train["patient_ids"])
train_image_data = sy.Tensor(train["images"]).private(min_val=0, max_val=255, data_subjects=data_subjects)
train_label_data = sy.Tensor(train["labels"]).private(min_val=0, max_val=5, data_subjects=data_subjects)

### Step 20: Make Validation data private

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
data_subjects = DataSubjectList.from_series(val["patient_ids"])
val_image_data = sy.Tensor(val["images"]).private(min_val=0, max_val=255, data_subjects=data_subjects)
val_label_data = sy.Tensor(val["labels"]).private(min_val=0, max_val=5, data_subjects=data_subjects)

### Step 21: Make Test data private

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
data_subjects = DataSubjectList.from_series(test["patient_ids"])
test_image_data = sy.Tensor(test["images"]).private(min_val=0, max_val=255, data_subjects=data_subjects)
test_label_data = sy.Tensor(test["labels"]).private(min_val=0, max_val=5, data_subjects=data_subjects)

### Step 22: Create Assets mapping

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
assets = {
    "train_images": train_image_data,
    "train_labels": train_label_data,
    "val_images": val_image_data,
    "val_labels": val_label_data,
    "test_images": test_image_data,
    "test_labels": test_label_data,
}

### Step 23: Set dataset name

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
dataset_name = "TissueMNIST"
print(dataset_name)

### Step 24: Set dataset description

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
dataset_description = ""
print(dataset_description)

### Step 25: Upload Dataset to the Domain node

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# upload the MedNIST data
domain_client.load_dataset(
    assets=assets, name=dataset_name, description=dataset_description
)

### Step 26: View Uploaded Datasets

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client.datasets

### Step 27: View assets of the uploaded dataset

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client.datasets[0]

### Step 28: 🖐 Raise your hand in Zoom

# BEGIN SECTION 5: JOIN A NETWORK

### Step 29: What is a Network ?

<img src="img/tab_info.png" alt="tab" style="width: 100px; margin:0;" />

A Network Node is a node that connects different domains to a broader base of data scientists also known as a network's members. It is a server which exists outside of any data owner's institution, providing services to the network of data owners and data scientists. In short, a Network node provides a secure interface between its cohorts or Domains and its members or Data Scientists.

For the scope of this demonstration, OpenMined has created a Network node called Ad Astra, to which we will register our Domain Node to.

### Step 30: Browse available Networks

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
sy.networks

### Step 31: Select a Network

In [ ]:
network_client = sy.networks["Ad Astra Demo 1"]

### Step 32: List the domains available on the selected Network

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
network_client.domains

### Step 33: Join the selected Network

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client.apply_to_network(network_client)

### Step 34: Verify if our domain is part of the Network

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
network_client.domains

### Step 35: 🖐 Raise your hand in Zoom

# BEGIN SECTION 6: CREATE USER

### Step 36: Create a Data Scientist User

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
ds_login_credentials = domain_client.create_user(name="Default Name", email="Default Email", password="Default Password")

### Step 37: Display DS login credentials

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
print(ds_login_credentials)

### Step 38: 🖐 Raise your hand in Zoom

# BEGIN SECTION 7: Conclusion

### Step 39: Submit Data Scientist Credentials

### Step 40: Fill Survey

### Step 41: Join the slack channel

# 🙌🏽 Notebook Complete!